In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read SpaceX data from .csv file (I re-read the ".../spacex_launch_dash.csv" file into a new df):
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

# To see the manes of the cols and so on I loaded the same .csv file on a Jupyter lab, like so:
# import pandas as pd
# df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
# df.head()


# Select minimum and maximum Payload mass for the range slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Now we create a dash application like we´ve done before:
app = dash.Dash(__name__)

# Create app layout:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style = {'TextAlign':'center', 'color':'#503D36', 'font-size':40}),
                                        
                                        # TASK 1: Add a Launch Site Drop-down Input Component
                                        dcc.Dropdown(id='site-dropdown', 
                                                    options = [{'label':'All Sites', 'value':'ALL'},
                                                                {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                                {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                                {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                                {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                                    value = 'ALL',
                                                    placeholder = 'Launch Site',
                                                    searchable = True),
                                        html.Br(),

                                        #TASK 2: Add a callback function to render 'success-pie-chart' based on selected site dropdown
                                        # This is, for the selected site, display a pir chart with the counts for successful VS non-successful launches.
                                        # AKA Class 1 VS Class 0
                                        html.Div(dcc.Graph(id='success-pie-chart')),
                                        html.Br(),

                                        # TASK 3: Add a range Slider to Select Payload
                                        dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                                        marks={0:'o', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                                        value=[min_payload, max_payload]),
                                        
                                        # TASK 4: Add a Callback function to render the 'success-payload-scatter-chart' scatter plot 
                                        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2: Add a callback function to render 'success-pie-chart' based on selected site dropdown.
# Here the input is 'site-dropdown' and the output should be the 'success-pie-chart'
# First we define the callback function:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)

# Now we define the pie chart:

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names = 'Launch Site', title = 'Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for. selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_df=filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(site_df, values='class count', names='class', title=f'Total Success Launches for Site: {entered_site}')
        return fig

# TASK 4: Add a Callback function to render the 'success-payload-scatter-chart' scatter plot
# First we defince the callback function:
# Here the inputs are two: both the 'site-dropdown' and the 'payload-slidersite-dropdown, while the output should be the 'success-payload-scatter-chart'
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')]
)

# Now we define the scatter plot:
def get_scatter_chart(entered_site, payload_slider):
    # Since the playload_slider gives a two value list(min and max), we need to select all rows which payload falls into this range
    low, high = payload_slider
    range_df=(spacex_df['Payload mass (kg)'] > low) and (spacex_df['Payload Mass (kg)'] < high)
    dropdown_scatter = spacex_df[range_df]
    # If 'ALL' sites are selected, render a scatter plot to display all values for the variable 'Payload Mass (kg)' and variable 'class'
    if entered_site == 'ALL':
        fig = px.scatter(dropdown_scatter, x='Payload Mass (kg)', y='class', hover_data=['Booster Version'],
                        color='Booster Version Category', title='Correlation between Payload and Success Rate for all launching sites')
        return fig
    # If a specific launching site has been selected, you need to filter the spacex_df first, and render a scatter chart to show values 
    # 'Payload Mass (kg)' and 'class' for the selected site, and color-label the points using 'Booster Version Category' as before.
    else:
        dropdown_scatter = dropdown_scatter[spacex_df['Launch Site'] == entered_site]
        fig = px.scatter( dropdown_scatter, x='Payload Mass (kg)', y='class', hover_data=['Booster Version'],
                        color='Booster Version Category', title=f'Correlation between launching success and Payload Mass for launching site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
